## Cell Pose Prediction Masks
Updated - 2020 Nov. 11

By - JC Vizcarra (SageBionetworks intern)

This notebook is an auxillary notebook for creating prediction nuclei masks to use in the nuclei segmentation interactors.

In [ ]:
import sys
sys.path.append('/data/')

from cellpose import utils, io, models
import large_image
import configs as ci
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from imageio import imwrite

In [ ]:
# read the ome.tif and use the first frame -- I assume this is the DAPI / DNA channel
ts = large_image.getTileSource(ci.OMETIF_FILEPATH2)

chnames = ts.getMetadata()['channelmap']
frame = chnames[list(chnames.keys())[0]]

kwargs = {'format': large_image.tilesource.TILE_FORMAT_NUMPY, 'frame': frame}
im = ts.getRegion(**kwargs)[0][:, :, 0]

In [ ]:
def cellpose_prediction(im, diameter, dim=256, save_path=None):    
    """Predict the nuclei mask for a very large image by reading it in small square regions. Note that this does not 
    have smart merging of nuclei that are cut between the regions. 
    
    Parameters
    ----------
    im : array-like
        image to run cellpose on for nuclei detection / segmentation, could be a very large image
    diameter : int
        diameter of the nuclei to use as a prior, the cellpose results vary on this
    dim : int (default: 256)
        the width and height of the regions to chop the image by for running cellpose on
    save_path : str (default: None)
        path to save mask to, if None then image is not saved
        
    Return
    ------
    mask : array-like
        the labeled mask with the nuclei predictions
        
    """
    # create the cellpose model for nuclei detection without GPU usage
    model = models.Cellpose(gpu=False, model_type='nuclei')    
    
    # read the image in chunks
    height, width = im.shape
    
    ims = []
    channels = []
    xys = []
    
    for x in range(0, width, dim):
        for y in range(0, height, dim):
            xys.append((x,y))
            ims.append(im[y:y+dim, x:x+dim])
            channels.append([0,0])

            
    # run cellpose on the chunks
    masks = model.eval(ims, diameter=diameter, channels=channels)[0]
    
    # use the masks to reconstruct the final image
    mask = np.zeros((height, width), np.int32)

    for m, xy in tqdm(zip(masks, xys)):
        # get max in final_mask
        max_value = np.max(mask)

        # sum the _max to all non-zero values
        m = m.copy()
        m[m!=0] += max_value

        # add this mask to its location in final mask
        mask[xy[1]:xy[1]+dim, xy[0]:xy[0]+dim] = m
        
    if save_path is not None:
        imwrite(save_path, mask)
        
    return mask


# cellpose_prediction(im, 16, dim=1000, save_path='/data/ome.tiffs/TonsilNucleiSeg/TONSIL-1_40X_16diameter_cellpose_nucleiMask.tif');
cellpose_prediction(im, 20, dim=1000, save_path='/data/ome.tiffs/TonsilNucleiSeg/TONSIL-1_40X_20diameter_cellpose_nucleiMask.tif');